# Checking a database

It has happened ocassionally that the reported data is not internally consistent. Here we show how to make the most of pyam's tools to check your database so you can double check the data in the IAM database. 

In [1]:
import time
from pprint import pprint

import pyam
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

<IPython.core.display.Javascript object>

We start with the tutorial data, it contains only a fraction of the AR5 data so is not internally consistent and is hence the perfect dataset to start with.

In [2]:
df = pyam.IamDataFrame(data='tutorial_AR5_data.csv', encoding='utf-8')

INFO:root:Reading `tutorial_AR5_data.csv`


In [3]:
df.head()

,model,scenario,region,variable,unit,year,value
0,AIM-Enduse 12.1,EMF27-450-Conv,ASIA,Emissions|CO2,Mt CO2/yr,2005,10540.74
3,AIM-Enduse 12.1,EMF27-450-Conv,LAM,Emissions|CO2,Mt CO2/yr,2005,3285.00
6,AIM-Enduse 12.1,EMF27-450-Conv,MAF,Emissions|CO2,Mt CO2/yr,2005,4302.21
9,AIM-Enduse 12.1,EMF27-450-Conv,OECD90,Emissions|CO2,Mt CO2/yr,2005,12085.85
12,AIM-Enduse 12.1,EMF27-450-Conv,REF,Emissions|CO2,Mt CO2/yr,2005,3306.95


## Summary

With the `pyam.IamDataFrame.check_internal_consistency` method, we can check the internal consistency of a database. If this method returns `None`, the database is internally consistent (i.e. the total variables are the sum of the sectoral breakdowns and the regional breakdown 

In the rest of this tutorial, we give you a chance to better understand this method. We go through what it is actually doing and show you the kind of output you can expect.

## Checking variables are the sum of their components

We are going to use the `check_aggregate` method of `IamDataFrame` to check that the components of a variable sum to its total. This method takes `np.is_close` arguments as keyword arguments, we show our recommended settings here.

In [4]:
np_isclose_args = {
    "equal_nan": True,
    "rtol": 1e-03,
    "atol": 1e-05,
}

Using `check_aggregate` on the `IamDataFrame` allows us to quickly check if a single variable is equal to the sum of its sectoral components (e.g. is `Emissions|CO2` equal to `Emissions|CO2|Transport` plus `Emissions|CO2|Solvents` plus `Emissions|CO2|Energy` etc.). A returned `DataFrame` will show us where the aggregate is not equal to the sum of components.

In [5]:
df.check_aggregate(
    "Emissions|CO2", 
    **np_isclose_args
)

INFO:root:Emissions|CO2 - 1368 of 1522 data points are not aggregates of components


2005      2010  \
variable      model           scenario            region                       
Emissions|CO2 AIM-Enduse 12.1 EMF27-450-Conv      ASIA    10540.74  13160.18   
                                                  LAM      3285.00   3294.54   
                                                  MAF      4302.21   4487.54   
                                                  OECD90  12085.85  12744.33   
                                                  REF      3306.95   3604.42   
                                                  World   34492.05  38321.78   
                              EMF27-450-NoCCS     ASIA    10540.74  13160.11   
                                                  LAM      3285.00   3286.68   
                                                  MAF      4302.21   4487.49   
                                                  OECD90  12085.85  12744.16   
                                                  REF      3306.95   3604.39   
                                                  World   34492.05  38313.59   
                              EMF27-550-LimBio    ASIA    10540.74  13160.11   
                                                  LAM      3285.00   3286.68   
                                                  MAF      4302.21   4487.49   
                                                  OECD90  12085.85  12744.16   
                                                  REF      3306.95   3604.39   
                                                  World   34492.05  38313.59   
                              EMF27-Base-FullTech ASIA    10540.74  13160.11   
                                                  LAM      3285.00   3286.68   
                                                  MAF      4302.21   4487.49   
                                                  OECD90  12085.85  12744.16   
                                                  REF      3306.95   3604.39   
                                                  World   34492.05  38313.59   
                              EMF27-G8-EERE       ASIA    10540.74  13152.56   
                                                  LAM      3285.00   3286.52   
                                                  MAF      4302.21   4487.02   
                                                  OECD90  12085.85  12750.81   
                                                  REF      3306.95   3596.74   
                                                  World   34492.05  38304.41   
...                                                            ...       ...   
              REMIND 1.5      EMF27-450-NoCCS     OECD90  15111.39  15254.16   
                                                  World   33837.41  38224.94   
                              EMF27-550-LimBio    ASIA    10193.98  13239.55   
                                                  LAM      2926.60   3478.79   
                                                  MAF      4035.32   4381.03   
                                                  OECD90  15111.39  15241.56   
                                                  World   33837.41  37970.11   
                              EMF27-Base-FullTech ASIA    10193.98  13478.78   
                                                  LAM      2926.60   3508.40   
                                                  MAF      4035.32   4381.09   
                                                  OECD90  15111.39  15234.63   
                                                  World   33837.41  38293.08   
              WITCH_EMF27     EMF27-450-Conv      ASIA     9895.45  13210.18   
                                                  LAM      4660.57   4644.17   
                                                  MAF      2508.31   2673.95   
                                                  OECD90  12644.40  12597.55   
                                                  REF      3870.58   4035.17   
                                                  World   3357

As we are missing most of the sectoral data in this subset of AR5, the total variables are mostly not equal to their components. The data table above shows us which model-scenario-region combinations this is the case for. As a user, we would then have to examine which sectors we have for each of these model-scenario-region combinations in order to determine what is missing.

### Checking multiple variables

We can then wrap this altogether to check all or a subset of the variables in an `IamDataFrame`.

In [6]:
for variable in df.filter(level=1).variables():
    diff = df.check_aggregate(
        variable, 
        **np_isclose_args
    )
    # you could then make whatever summary you wanted
    # with diff

INFO:root:Emissions|CO2 - 1368 of 1522 data points are not aggregates of components
INFO:root:Price|Carbon - cannot check aggregate because it has no components
INFO:root:Primary Energy|Coal - cannot check aggregate because it has no components


The output tells us where there are issues as well as where it is not possible to actually check sums because no components have been reported. 

## Checking that regions sum to aggregate regions

Similarly to checking that the sum of a variable's components give the declared total, we can check that summing regions gives the intended total.

To do this, we use the `check_aggregate_regions` method of `IamDataFrame`. By default, this method checks that all the regions in the dataframe sum to World. 

Using `check_aggregate_regions` on the `IamDataFrame` allows us to quickly check if a regional total for a single variable is equal to the sum of its regional contributors. A returned `DataFrame` will show us where the aggregate is not equal to the sum of components.

In [7]:
df.check_aggregate_regions(
    "Emissions|CO2",
    **np_isclose_args
)

INFO:root:Emissions|CO2 - 503 of 503 data points are not aggregates of regional components


2005      2010  \
region model           scenario            variable                            
World  AIM-Enduse 12.1 EMF27-450-Conv      Emissions|CO2  34492.05  38321.78   
                       EMF27-450-NoCCS     Emissions|CO2  34492.05  38313.59   
                       EMF27-550-LimBio    Emissions|CO2  34492.05  38313.59   
                       EMF27-Base-FullTech Emissions|CO2  34492.05  38313.59   
                       EMF27-G8-EERE       Emissions|CO2  34492.05  38304.41   
       GCAM 3.0        AMPERE3-450         Emissions|CO2  31473.40  31678.13   
                       AMPERE3-450P-CE     Emissions|CO2  31473.40  31678.14   
                       AMPERE3-450P-EU     Emissions|CO2  31473.40  31678.14   
                       AMPERE3-550         Emissions|CO2  31473.40  31678.13   
                       AMPERE3-Base-EUback Emissions|CO2  31473.40  31678.13   
                       AMPERE3-CF450P-EU   Emissions|CO2  31473.40  31678.13   
                       AMPERE3-RefPol      Emissions|CO2  31473.40  31678.14   
                       EMF27-450-Conv      Emissions|CO2  31473.62  34700.97   
                       EMF27-450-NoCCS     Emissions|CO2  31473.62  34685.29   
                       EMF27-550-LimBio    Emissions|CO2  31473.62  34685.29   
                       EMF27-Base-FullTech Emissions|CO2  31473.62  34685.29   
       IMAGE 2.4       AMPERE3-450         Emissions|CO2  34111.29  35344.27   
                       AMPERE3-450P-CE     Emissions|CO2  34111.14  35343.93   
                       AMPERE3-450P-EU     Emissions|CO2  34111.14  35343.93   
                       AMPERE3-550         Emissions|CO2  34111.29  35318.50   
                       AMPERE3-RefPol      Emissions|CO2  34124.32  35746.98   
                       EMF27-550-LimBio    Emissions|CO2  34259.27  35594.34   
                       EMF27-Base-FullTech Emissions|CO2  34391.39  35246.31   
       MERGE_EMF27     EMF27-450-Conv      Emissions|CO2  28501.22  33303.16   
                       EMF27-550-LimBio    Emissions|CO2  28501.22  33303.16   
                       EMF27-Base-FullTech Emissions|CO2  28501.22  33303.16   
                       EMF27-G8-EERE       Emissions|CO2  28501.22  33303.16   
       MESSAGE V.4     AMPERE3-450         Emissions|CO2  34474.59  36035.69   
                       AMPERE3-450P-EU     Emissions|CO2  34474.49  36036.02   
                       AMPERE3-550         Emissions|CO2  34474.51  36035.96   
                       AMPERE3-RefPol      Emissions|CO2  34474.46  36035.98   
                       EMF27-550-LimBio    Emissions|CO2  34491.02  36087.09   
                       EMF27-Base-FullTech Emissions|CO2  34491.02  36087.09   
       REMIND 1.5      AMPERE3-450         Emissions|CO2  33841.49  37365.91   
                       AMPERE3-450P-CE     Emissions|CO2  33841.49  37357.58   
                       AMPERE3-450P-EU     Emissions|CO2  33841.49  37356.37   
                       AMPERE3-550         Emissions|CO2  33841.49  37366.12   
                       AMPERE3-550P-EU     Emissions|CO2  33841.49  37360.58   
                       AMPERE3-Base-EUback Emissions|CO2  33841.49  37371.87   
                       AMPERE3-CF450P-EU   Emissions|CO2  33841.49  37365.98   
                       AMPERE3-RefPol      Emissions|CO2  33841.49  37372.53   
                       EMF27-450-Conv      Emissions|CO2  33837.41  37977.31   
                       EMF27-450-NoCCS     Emissions|CO2  33837.41  38224.94   
                       EMF27-550-LimBio    Emissions|CO2  33837.41  37970.11   
                       EMF27-Base-FullTech Emissions|CO2  33837.41  38293.08   
       WITCH_EMF27     EMF27-450-Conv      Emissions|CO2  33579.32  37161.03   
                       EMF27-550-LimBio    Emissions|CO2  33582.45  37181.70   
                       EMF27-Base-FullTech Emissions|CO2  33567.64  37320.72   

                                                             

Again, as the AR5 snapshot is incomplete, all World sums are not equal to the regions provided.

Once again, we can repeat this analysis over all the variables of interest in an `IamDataFrame`.

In [8]:
for variable in df.variables():
    diff = df.check_aggregate_regions(
        variable, 
        **np_isclose_args
    )
    # you could then make whatever summary you wanted
    # with diff
    if diff is not None:
        eg = diff

eg.head(20)

INFO:root:Emissions|CO2 - 503 of 503 data points are not aggregates of regional components
INFO:root:Emissions|CO2|Fossil Fuels and Industry - 239 of 239 data points are not aggregates of regional components
INFO:root:Emissions|CO2|Fossil Fuels and Industry|Energy Supply - cannot check regional aggregate because it has no regional components
INFO:root:Emissions|CO2|Fossil Fuels and Industry|Energy Supply|Electricity - cannot check regional aggregate because it has no regional components
INFO:root:Price|Carbon - cannot check regional aggregate because it has no regional components
INFO:root:Primary Energy - 502 of 503 data points are not aggregates of regional components
INFO:root:Primary Energy|Coal - cannot check regional aggregate because it has no regional components
INFO:root:Primary Energy|Fossil|w/ CCS - cannot check regional aggregate because it has no regional components
INFO:root:Temperature|Global Mean|MAGICC6|MED - cannot check regional aggregate because it has no regional c

2005    2010  \
region model           scenario            variable                         
World  AIM-Enduse 12.1 EMF27-450-Conv      Primary Energy  458.20  518.89   
                       EMF27-450-NoCCS     Primary Energy  458.20  518.81   
                       EMF27-550-LimBio    Primary Energy  458.20  518.81   
                       EMF27-Base-FullTech Primary Energy  458.20  518.81   
                       EMF27-G8-EERE       Primary Energy  458.20  518.64   
       GCAM 3.0        AMPERE3-450         Primary Energy  460.41  504.35   
                       AMPERE3-450P-CE     Primary Energy  460.41  504.35   
                       AMPERE3-450P-EU     Primary Energy  460.41  504.35   
                       AMPERE3-550         Primary Energy  458.03  501.89   
                       AMPERE3-Base-EUback Primary Energy  457.76  501.61   
                       AMPERE3-CF450P-EU   Primary Energy  460.41  504.35   
                       AMPERE3-RefPol      Primary Energy  457.76  501.61   
                       EMF27-450-Conv      Primary Energy  458.82  504.59   
                       EMF27-450-NoCCS     Primary Energy  458.82  504.45   
                       EMF27-550-LimBio    Primary Energy  458.82  504.45   
                       EMF27-Base-FullTech Primary Energy  458.82  504.45   
       IMAGE 2.4       AMPERE3-450         Primary Energy  441.25  473.91   
                       AMPERE3-450P-CE     Primary Energy  441.25  473.91   
                       AMPERE3-450P-EU     Primary Energy  441.25  473.91   
                       AMPERE3-550         Primary Energy  441.25  473.79   

                                                             2020    2030  \
region model           scenario            variable                         
World  AIM-Enduse 12.1 EMF27-450-Conv      Primary Energy  500.15  521.23   
                       EMF27-450-NoCCS     Primary Energy  500.24  493.64   
                       EMF27-550-LimBio    Primary Energy  544.28  592.53   
                       EMF27-Base-FullTech Primary Energy  545.24  619.43   
                       EMF27-G8-EERE       Primary Energy  487.22  463.48   
       GCAM 3.0        AMPERE3-450         Primary Energy  618.51  743.09   
                       AMPERE3-450P-CE     Primary Energy  618.74  753.34   
                       AMPERE3-450P-EU     Primary Energy  624.50  769.43   
                       AMPERE3-550         Primary Energy  622.66  751.83   
                       AMPERE3-Base-EUback Primary Energy  632.27  788.66   
                       AMPERE3-CF450P-EU   Primary Energy  635.96  793.04   
                       AMPERE3-RefPol      Primary Energy  622.68  764.56   
                       EMF27-450-Conv      Primary Energy  556.01  630.79   
                       EMF27-450-NoCCS     Primary Energy  546.22  599.28   
                       EMF27-550-LimBio    Primary Energy  576.89  663.16   
                       EMF27-Base-FullTech Primary Energy  613.10  730.03   
       IMAGE 2.4       AMPERE3-450         Primary Energy  544.13  577.42   
                       AMPERE3-450P-CE     Primary Energy  580.13  654.46   
                       AMPERE3-450P-EU     Primary Energy  598.26  696.60   
                       AMPERE3-550         Primary Energy  559.87  603.66   

                                                             2040     2050  \
region model           scenario            variable                          
World  AIM-Enduse 12.1 EMF27-450-Conv      Primary Energy  569.53   581.44   
                       EMF27-450-NoCCS     Primary Energy  583.82   614.23   
                       EMF27-550-LimBio    Primary Energy  639.70   679.98   
                       EMF27-Base-FullTech Primary Energy  715.12   816.88   
                       EMF27-G8-EERE       Primary Energy  499.48   555.22   
       GCAM 3.0        AMPERE3-450         Primary Energy  848.71   935.92   
                       AMPERE3-450P-CE     Prim

## An internally consistent database

If we have an internally consistent database, the returned `DataFrame` will always be none. 

Repeating the same analysis as above can then confirm that all is well with the database as well as give us some insight into which variables do not have regional or sectoral breakdowns reported.

In [9]:
consistent_df = pyam.IamDataFrame(data="tutorial_check_database.csv", encoding='utf-8')

INFO:root:Reading `tutorial_check_database.csv`


In [10]:
for variable in consistent_df.filter(level=1).variables():
    diff = consistent_df.check_aggregate(
        variable, 
        **np_isclose_args
    )
    assert diff is None

INFO:root:Primary Energy|Coal - cannot check aggregate because it has no components
INFO:root:Primary Energy|Gas - cannot check aggregate because it has no components
INFO:root:Emissions|CF4 - cannot check aggregate because it has no components
INFO:root:Emissions|CH4 - cannot check aggregate because it has no components


In [11]:
for variable in consistent_df.filter(level=1).variables():
    diff = consistent_df.check_aggregate_regions(
        variable, 
        **np_isclose_args
    )
    assert diff is None

INFO:root:Emissions|C2F6 - cannot check regional aggregate because it has no regional components
INFO:root:Emissions|CF4 - cannot check regional aggregate because it has no regional components


## Putting it altogether

Finally, we provide the `check_internal_consistency` method which does all the above for you and returns a dictionary with all of the dataframes which document the errors.

Note: at the moment, this method's regional checking is limited to checking that all the regions sum to the World region. We cannot make this more automatic unless we start to store how the regions relate, see [this issue](https://github.com/IAMconsortium/pyam/issues/106). 

In [12]:
# if all is good, None is returned
print("Checking consistent data"); time.sleep(0.5)
assert consistent_df.check_internal_consistency() is None

# otherwise we get a dict back
print("Checking AR5 subset"); time.sleep(0.5)
errors = df.check_internal_consistency()

Checking consistent data


INFO:root:Emissions|CO2|Cars - cannot check aggregate because it has no components
INFO:root:Emissions|CO2|Power - cannot check aggregate because it has no components
INFO:root:Primary Energy|Coal - cannot check aggregate because it has no components
INFO:root:Primary Energy|Gas - cannot check aggregate because it has no components
INFO:root:Emissions|C2F6 - cannot check regional aggregate because it has no regional components
INFO:root:Emissions|C2F6|Industry - cannot check aggregate because it has no components
INFO:root:Emissions|C2F6|Industry - cannot check regional aggregate because it has no regional components
INFO:root:Emissions|C2F6|Solvents - cannot check aggregate because it has no components
INFO:root:Emissions|C2F6|Solvents - cannot check regional aggregate because it has no regional components
INFO:root:Emissions|CF4 - cannot check aggregate because it has no components
INFO:root:Emissions|CF4 - cannot check regional aggregate because it has no regional components
INFO:ro

Checking AR5 subset


INFO:root:Emissions|CO2 - 1390 of 1522 data points are not aggregates of components
INFO:root:Emissions|CO2 - 503 of 503 data points are not aggregates of regional components
INFO:root:Emissions|CO2|Fossil Fuels and Industry - 1258 of 1258 data points are not aggregates of components
INFO:root:Emissions|CO2|Fossil Fuels and Industry - 239 of 239 data points are not aggregates of regional components
INFO:root:Emissions|CO2|Fossil Fuels and Industry|Energy Supply - 239 of 239 data points are not aggregates of components
INFO:root:Emissions|CO2|Fossil Fuels and Industry|Energy Supply - cannot check regional aggregate because it has no regional components
INFO:root:Emissions|CO2|Fossil Fuels and Industry|Energy Supply|Electricity - cannot check aggregate because it has no components
INFO:root:Emissions|CO2|Fossil Fuels and Industry|Energy Supply|Electricity - cannot check regional aggregate because it has no regional components
INFO:root:Price|Carbon - cannot check aggregate because it has

In [13]:
pprint([k for k in errors.keys()])

['Emissions|CO2-aggregate',
 'Emissions|CO2-regional',
 'Emissions|CO2|Fossil Fuels and Industry-aggregate',
 'Emissions|CO2|Fossil Fuels and Industry-regional',
 'Emissions|CO2|Fossil Fuels and Industry|Energy Supply-aggregate',
 'Primary Energy-aggregate',
 'Primary Energy-regional']


In [14]:
errors["Emissions|CO2-aggregate"]

2005      2010  \
variable      model           scenario            region                       
Emissions|CO2 AIM-Enduse 12.1 EMF27-450-Conv      ASIA    10540.74  13160.18   
                                                  LAM      3285.00   3294.54   
                                                  MAF      4302.21   4487.54   
                                                  OECD90  12085.85  12744.33   
                                                  REF      3306.95   3604.42   
                                                  World   34492.05  38321.78   
                              EMF27-450-NoCCS     ASIA    10540.74  13160.11   
                                                  LAM      3285.00   3286.68   
                                                  MAF      4302.21   4487.49   
                                                  OECD90  12085.85  12744.16   
                                                  REF      3306.95   3604.39   
                                                  World   34492.05  38313.59   
                              EMF27-550-LimBio    ASIA    10540.74  13160.11   
                                                  LAM      3285.00   3286.68   
                                                  MAF      4302.21   4487.49   
                                                  OECD90  12085.85  12744.16   
                                                  REF      3306.95   3604.39   
                                                  World   34492.05  38313.59   
                              EMF27-Base-FullTech ASIA    10540.74  13160.11   
                                                  LAM      3285.00   3286.68   
                                                  MAF      4302.21   4487.49   
                                                  OECD90  12085.85  12744.16   
                                                  REF      3306.95   3604.39   
                                                  World   34492.05  38313.59   
                              EMF27-G8-EERE       ASIA    10540.74  13152.56   
                                                  LAM      3285.00   3286.52   
                                                  MAF      4302.21   4487.02   
                                                  OECD90  12085.85  12750.81   
                                                  REF      3306.95   3596.74   
                                                  World   34492.05  38304.41   
...                                                            ...       ...   
              REMIND 1.5      EMF27-450-NoCCS     OECD90  15111.39  15254.16   
                                                  World   33837.41  38224.94   
                              EMF27-550-LimBio    ASIA    10193.98  13239.55   
                                                  LAM      2926.60   3478.79   
                                                  MAF      4035.32   4381.03   
                                                  OECD90  15111.39  15241.56   
                                                  World   33837.41  37970.11   
                              EMF27-Base-FullTech ASIA    10193.98  13478.78   
                                                  LAM      2926.60   3508.40   
                                                  MAF      4035.32   4381.09   
                                                  OECD90  15111.39  15234.63   
                                                  World   33837.41  38293.08   
              WITCH_EMF27     EMF27-450-Conv      ASIA     9895.45  13210.18   
                                                  LAM      4660.57   4644.17   
                                                  MAF      2508.31   2673.95   
                                                  OECD90  12644.40  12597.55   
                                                  REF      3870.58   4035.17   
                                                  World   3357